In [ ]:
import ollama

# 풀할 모델 리스트
models_to_pull = [
    "llama3.1:8b",
    "gemma2:27b",
    "mistral:7b",
    "llama3.2:3b"
]

# 모델 풀하기
for model_name in models_to_pull:
    print(f"Pulling model: {model_name}")
    ollama.pull(model_name)

print("Selected models have been pulled successfully.")

In [2]:
import pandas as pd
import os
import re
import ollama

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# 모델 이름 리스트 자동으로 가져오기
models_data = ollama.list()
models = [model['name'] for model in models_data['models']]  # 모델 이름 추출

def extract_sentiment(text):
    keywords = ["긍정", "부정", "중립"]
    # 텍스트를 소문자로 변환하고 키워드가 가장 먼저 등장하는 순서를 판별
    text = str(text).lower()
    indices = {keyword: text.find(keyword) for keyword in keywords}
    # 등장한 키워드 중 가장 빠른 순서의 키워드 반환 (등장하지 않으면 기본값 '중립')
    earliest = min((index for index in indices.values() if index != -1), default=-1)
    for keyword, index in indices.items():
        if index == earliest:
            return keyword
    return "중립"

# 각 모델에 대해 sentiment 분석 수행
for model_name in models:
    # 모델별 CSV 파일 경로 설정 (특수 문자 대체)
    sanitized_model_name = re.sub(r'[^a-zA-Z0-9_-]', '_', model_name)
    output_csv = os.path.join(output_folder, f'{sanitized_model_name}_NOCoT.csv')
    
    # Load or create the DataFrame
    if os.path.exists(output_csv):
        news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
    else:
        news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
        news_data['Sentiment Analysis'] = None  # Initialize with None for new column

    # Loop through each article and ask for sentiment analysis only if 'Sentiment Analysis' is empty
    for index, row in news_data.iterrows():
        if pd.isna(row['Sentiment Analysis']):  # Check if the sentiment analysis is missing
            # Define the system message to set the role
            system_message = {
                'role': 'system',
                'content': "당신은 금융 뉴스 기사에서 감정을 평가하는 주식 분석가입니다."
            }

            # Define the question based on the article title and provide the response format
            question = f"'{row['Title']}' 제목의 {row['Company']}에 대한 톤이 긍정적, 부정적 또는 중립적인지 결정하십시오.\n" \
                       "다음 형식으로 응답해 주세요. 이유는 생략합니다:\n" \
                       "Sentiment: [긍정/부정/중립]"

            # Send the question to the model
            response = ollama.chat(model=model_name, messages=[
                system_message,
                {
                    'role': 'user',
                    'content': question,
                },
            ])

            # Extract the response content
            sentiment_response = response['message']['content'] if 'message' in response and 'content' in response['message'] else "No response"

            # Update the sentiment analysis directly in the DataFrame
            news_data.at[index, 'Sentiment Analysis'] = sentiment_response

            # Save the DataFrame back to CSV after each response
            news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

    # Apply the sentiment extraction function to create the Predict column
    news_data['Predict'] = news_data['Sentiment Analysis'].apply(extract_sentiment)
    
    # Save the updated DataFrame with the Predict column back to the CSV file
    news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 모델의 감정 분석이 완료되었습니다.")


메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_NOCoT.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/gemma2_27b_NOCoT.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/mistral_7b_NOCoT.csv'에 저장되었습니다.
메시지 전송 성공: llama3.2:3b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_2_3b_NOCoT.csv'에 저장되었습니다.
모든 모델의 감정 분석이 완료되었습니다.


In [4]:
import pandas as pd
import os
import re
import ollama

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# 모델 이름 리스트 자동으로 가져오기
models_data = ollama.list()
models = [model['name'] for model in models_data['models']]  # 모델 이름 추출

def extract_sentiment(text):
    keywords = ["긍정", "부정", "중립"]
    # 텍스트를 소문자로 변환하고 키워드가 가장 먼저 등장하는 순서를 판별
    text = str(text).lower()
    indices = {keyword: text.find(keyword) for keyword in keywords}
    # 등장한 키워드 중 가장 빠른 순서의 키워드 반환 (등장하지 않으면 기본값 '중립')
    earliest = min((index for index in indices.values() if index != -1), default=-1)
    for keyword, index in indices.items():
        if index == earliest:
            return keyword
    return "중립"

# 각 모델에 대해 sentiment 분석 수행
for model_name in models:
    # 모델별 CSV 파일 경로 설정 (특수 문자 대체)
    sanitized_model_name = re.sub(r'[^a-zA-Z0-9_-]', '_', model_name)
    output_csv = os.path.join(output_folder, f'{sanitized_model_name}_CoT.csv')
    
    # Load or create the DataFrame
    if os.path.exists(output_csv):
        news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
    else:
        news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
        news_data['Sentiment Analysis'] = None  # Initialize with None for new column

    # Loop through each article and ask for sentiment analysis only if 'Sentiment Analysis' is empty
    for index, row in news_data.iterrows():
        if pd.isna(row['Sentiment Analysis']):  # Check if the sentiment analysis is missing
            # Define the system message to set the role
            system_message = {
                'role': 'system',
                'content': "당신은 금융 뉴스 기사에서 감정을 평가하는 주식 분석가입니다."
            }

            # Define the question based on the article title and provide the response format
            question = f"'{row['Title']}' 제목의 {row['Company']}에 대한 톤이 긍정적, 부정적 또는 중립적인지 결정하십시오.\n" \
                       "다음 형식으로 응답해 주세요.:\n" \
                       "Sentiment: [긍정/부정/중립]\n" \
                       "Reason: [기사를 기반으로 한 간략한 설명]"

            # Send the question to the model
            response = ollama.chat(model=model_name, messages=[
                system_message,
                {
                    'role': 'user',
                    'content': question,
                },
            ])

            # Extract the response content
            sentiment_response = response['message']['content'] if 'message' in response and 'content' in response['message'] else "No response"

            # Update the sentiment analysis directly in the DataFrame
            news_data.at[index, 'Sentiment Analysis'] = sentiment_response

            # Save the DataFrame back to CSV after each response
            news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

    # Apply the sentiment extraction function to create the Predict column
    news_data['Predict'] = news_data['Sentiment Analysis'].apply(extract_sentiment)
    
    # Save the updated DataFrame with the Predict column back to the CSV file
    news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 모델의 감정 분석이 완료되었습니다.")


메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_CoT.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/gemma2_27b_CoT.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/mistral_7b_CoT.csv'에 저장되었습니다.
메시지 전송 성공: llama3.2:3b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_2_3b_CoT.csv'에 저장되었습니다.
모든 모델의 감정 분석이 완료되었습니다.


In [5]:
import pandas as pd
import os
import re
from collections import Counter
import ollama

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# 모델 이름 리스트 자동으로 가져오기
models_data = ollama.list()
models = [model['name'] for model in models_data['models']]  # 모델 이름 추출

def extract_sentiment(text):
    keywords = ["긍정", "부정", "중립"]
    text = str(text).lower()
    indices = {keyword: text.find(keyword) for keyword in keywords}
    earliest = min((index for index in indices.values() if index != -1), default=-1)
    for keyword, index in indices.items():
        if index == earliest:
            return keyword
    return "중립"

def majority_vote(sentiments):
    """
    Determine the final sentiment based on majority voting.
    If no sentiment appears at least 3 times, return '중립'.
    """
    sentiment_counts = Counter(sentiments)
    for sentiment, count in sentiment_counts.items():
        if count >= 3:
            return sentiment
    return "중립"

# 각 모델에 대해 sentiment 분석 수행
for model_name in models:
    sanitized_model_name = re.sub(r'[^a-zA-Z0-9_-]', '_', model_name)
    output_csv = os.path.join(output_folder, f'{sanitized_model_name}_Bootstrap.csv')
    
    if os.path.exists(output_csv):
        news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
    else:
        news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
        news_data['Sentiment Analysis'] = None  # Initialize for 5 repetitions
        news_data['Predict'] = None  # Initialize final prediction column

    for index, row in news_data.iterrows():
        if pd.isna(row['Predict']):  # Skip if already processed
            sentiments = []
            for _ in range(5):  # Repeat 5 times
                system_message = {
                    'role': 'system',
                    'content': "당신은 금융 뉴스 기사에서 감정을 평가하는 주식 분석가입니다."
                }
                question = f"'{row['Title']}' 제목의 {row['Company']}에 대한 톤이 긍정적, 부정적 또는 중립적인지 결정하십시오.\n" \
                           "다음 형식으로 응답해 주세요. 이유는 생략합니다:\n" \
                           "Sentiment: [긍정/부정/중립]"
                response = ollama.chat(model=model_name, messages=[
                    system_message,
                    {'role': 'user', 'content': question},
                ])
                sentiment_response = response['message']['content'] if 'message' in response and 'content' in response['message'] else "중립"
                extracted_sentiment = extract_sentiment(sentiment_response)
                sentiments.append(extracted_sentiment)

            # Determine final sentiment based on majority vote
            final_sentiment = majority_vote(sentiments)
            news_data.at[index, 'Predict'] = final_sentiment

            # Save after each prediction
            news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 모델의 감정 분석이 완료되었습니다.")


메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_Bootstrap.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/gemma2_27b_Bootstrap.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/mistral_7b_Bootstrap.csv'에 저장되었습니다.
메시지 전송 성공: llama3.2:3b 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/llama3_2_3b_Bootstrap.csv'에 저장되었습니다.
모든 모델의 감정 분석이 완료되었습니다.


In [6]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# KR-FinBERT 모델 초기화
kr_finbert_model_name = "snunlp/KR-FinBert-SC"  # KR-FinBERT 모델 이름 (Hugging Face에 제공된 모델)
tokenizer = AutoTokenizer.from_pretrained(kr_finbert_model_name)
model = AutoModelForSequenceClassification.from_pretrained(kr_finbert_model_name, num_labels=3)
sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# 점수를 감정 레이블로 매핑
def map_label_to_sentiment(scores):
    labels = ["부정", "중립", "긍정"]
    max_index = scores.index(max(scores))
    return labels[max_index]

# 감정 분석 수행
output_csv = os.path.join(output_folder, 'KRFinBERT.csv')

if os.path.exists(output_csv):
    news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
else:
    news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
    news_data['Predict'] = None

for index, row in news_data.iterrows():
    if pd.isna(row['Predict']):  # 감정 분석 결과가 없는 경우
        text_to_analyze = row['Title']  # Title만 분석
        response = sentiment_pipeline(text_to_analyze)[0]  # 각 감정 점수 반환
        scores = [score['score'] for score in response]
        sentiment = map_label_to_sentiment(scores)

        # 감정 분석 결과 저장
        news_data.at[index, 'Predict'] = sentiment

        # CSV에 저장
        news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 데이터에 대해 감정 분석이 완료되었습니다.")

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/294k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/406M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


메시지 전송 성공: KR-FinBERT 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/KRFinBERT.csv'에 저장되었습니다.
모든 데이터에 대해 감정 분석이 완료되었습니다.


In [24]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
model_folder = './kb-albert-char-base-v2'  # 로컬 모델 폴더 경로
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# kb-albert-char-base-v2 모델 초기화 (로컬 폴더 사용)
try:
    tokenizer = AutoTokenizer.from_pretrained(model_folder)
    model = AutoModelForSequenceClassification.from_pretrained(model_folder, num_labels=3)
    sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)
    print("모델 로드 성공")
except Exception as e:
    print(f"모델 로드 실패: {e}")
    raise e

# 점수를 감정 레이블로 매핑
def map_label_to_sentiment(scores):
    # LABEL 순서: 긍정, 중립, 부정
    labels = ["긍정", "중립", "부정"]
    max_index = scores.index(max(scores))  # 가장 높은 점수의 인덱스를 찾음
    return labels[max_index]

# 감정 분석 수행
output_csv = os.path.join(output_folder, 'KBAlbert.csv')

if os.path.exists(output_csv):
    news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
    print("기존 CSV 파일 로드 성공")
else:
    news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
    news_data['Predict'] = None  # Predict 열 추가
    print("새 CSV 파일 로드 성공")

try:
    for index, row in news_data.iterrows():
        if pd.isna(row['Predict']):  # 감정 분석 결과가 없는 경우
            text_to_analyze = row['Title']  # Title 열 분석
            response = sentiment_pipeline(text_to_analyze)[0]  # 감정 점수 반환
            scores = [score['score'] for score in response]
            sentiment = map_label_to_sentiment(scores)  # 감정을 레이블로 매핑

            # 감정 분석 결과 저장
            news_data.at[index, 'Predict'] = sentiment

            # 중간 결과 저장 (주기적으로 저장)
            if index % 10 == 0:  # 10개 단위로 저장
                news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')
    
    # 최종 결과 저장
    news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"결과 저장 완료: {output_csv}")
except Exception as e:
    print(f"감정 분석 도중 에러 발생: {e}")
    raise e

print("모든 데이터에 대해 감정 분석이 완료되었습니다.")

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ./kb-albert-char-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


모델 로드 성공
기존 CSV 파일 로드 성공
결과 저장 완료: Sentiment/KBAlbert.csv
메시지 전송 성공: KB-ALBERT 모델로 Title 기반 감정 분석이 완료되었습니다. 파일은 'Sentiment/KBAlbert.csv'에 저장되었습니다.
모든 데이터에 대해 감정 분석이 완료되었습니다.


In [44]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# CSV 파일 경로 설정
input_csv = os.path.join(data_folder, 'news_data.csv')

# KcBERT 모델 초기화
kcbert_model_name = "beomi/kcbert-base"  # KcBERT 모델 이름 (Hugging Face에 제공된 모델)
tokenizer = AutoTokenizer.from_pretrained(kcbert_model_name)
model = AutoModelForSequenceClassification.from_pretrained(kcbert_model_name, num_labels=3)
sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# 점수를 감정 레이블로 매핑
def map_label_to_sentiment(scores):
    labels = ["부정", "중립", "긍정"]
    max_index = scores.index(max(scores))
    return labels[max_index]

# 감정 분석 수행
output_csv = os.path.join(output_folder, 'KCBERT.csv')

if os.path.exists(output_csv):
    news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
else:
    news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
    news_data['Predict'] = None

for index, row in news_data.iterrows():
    if pd.isna(row['Predict']):  # 감정 분석 결과가 없는 경우
        text_to_analyze = row['Title']  # Title만 분석
        response = sentiment_pipeline(text_to_analyze)  # 각 감정 점수 반환
        scores = [score['score'] for score in response[0]]
        sentiment = map_label_to_sentiment(scores)

        # 감정 분석 결과 저장
        news_data.at[index, 'Predict'] = sentiment

        # CSV에 저장
        news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 데이터에 대해 감정 분석이 완료되었습니다.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


메시지 전송 성공: KcBERT 모델의 감정 분석이 완료되었습니다. 파일은 'Sentiment/KCBERT.csv'에 저장되었습니다.
모든 데이터에 대해 감정 분석이 완료되었습니다.


In [1]:
import pandas as pd
import os
import re
import ollama

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# 참조 모델 리스트
reference_models = ["KCBERT", "KRFinBERT", "KBAlbert"]

# Ollama API를 사용하여 모델 이름 리스트 가져오기
models_data = ollama.list()
models = [model['name'] for model in models_data['models']]  # 모델 이름 추출

def extract_sentiment(text):
    keywords = ["긍정", "부정", "중립"]
    text = str(text).lower()
    indices = {keyword: text.find(keyword) for keyword in keywords}
    earliest = min((index for index in indices.values() if index != -1), default=-1)
    for keyword, index in indices.items():
        if index == earliest:
            return keyword
    return "중립"

# 각 모델 및 참조 모델에 대해 sentiment 분석 수행
for model_name in models:
    sanitized_model_name = re.sub(r'[^a-zA-Z0-9_-]', '_', model_name)
    
    for ref_model in reference_models:
        # 참조 모델 데이터 파일 경로
        reference_file = os.path.join(data_folder, f'{ref_model}.csv')
        
        # 참조 모델 데이터 로드
        if os.path.exists(reference_file):
            ref_df = pd.read_csv(reference_file, encoding='utf-8-sig')
        else:
            ref_df = pd.DataFrame(columns=['Title', 'Predict'])  # 빈 데이터프레임 초기화

        # 모델별, 참조 모델별 CSV 파일 경로 설정
        output_csv = os.path.join(output_folder, f'{sanitized_model_name}_{ref_model}-ICL.csv')
        
        # Load or create the DataFrame
        if os.path.exists(output_csv):
            news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
        else:
            news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
            news_data['Sentiment Analysis'] = None  # Initialize with None for new column

        # Loop through each article and ask for sentiment analysis only if 'Sentiment Analysis' is empty
        for index, row in news_data.iterrows():
            if pd.isna(row['Sentiment Analysis']):  # Check if the sentiment analysis is missing
                ref_predict = None
                ref_row = ref_df[ref_df['Title'] == row['Title']]
                if not ref_row.empty:
                    ref_predict = ref_row.iloc[0]['Predict']
                    
                # Define the system message to set the role
                system_message = {
                    'role': 'system',
                    'content': "당신은 금융 뉴스 기사에서 감정을 평가하는 주식 분석가입니다."
                }

                # Define the question based on the article title and provide the response format
                question = f"'{row['Title']}' 제목의 {row['Company']}에 대한 톤이 긍정적, 부정적 또는 중립적인지 결정하십시오.\n" \
                           f"참고로, {ref_model} 모델의 예측은 {ref_predict}입니다.\n" \
                           "다음 형식으로 응답해 주세요\n" \
                           "Sentiment: [긍정/부정/중립]\n" \
                           "Reason: [참고 모델과 다른 경우 간단한 이유를 작성해 주세요.]"

                # Send the question to the model
                response = ollama.chat(model=model_name, messages=[
                    system_message,
                    {
                        'role': 'user',
                        'content': question,
                    },
                ])

                # Extract the response content
                sentiment_response = response['message']['content'] if 'message' in response and 'content' in response['message'] else "No response"

                # Update the sentiment analysis directly in the DataFrame
                news_data.at[index, 'Sentiment Analysis'] = sentiment_response

                # Save the DataFrame back to CSV after each response
                news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

        # Apply the sentiment extraction function to create the Predict column
        news_data['Predict'] = news_data['Sentiment Analysis'].apply(extract_sentiment)
        
        # Save the updated DataFrame with the Predict column back to the CSV file
        news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 모델의 감정 분석이 완료되었습니다.")


메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KCBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KRFinBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KBAlbert 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KBAlbert-ICL.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/gemma2_27b_KCBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/gemma2_27b_KRFinBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: gemma2:27b 모델의 감정 분석이 KBAlbert 참조 기반으로 완료되었습니다. 파일은 'Sentiment/gemma2_27b_KBAlbert-ICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KCBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KRFinBERT-ICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KBAlbert 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KBAlbert-IC

In [ ]:
import pandas as pd
import os
import re
import ollama
from collections import Counter

# 폴더 경로 설정
data_folder = 'Data'
output_folder = 'Sentiment'
os.makedirs(output_folder, exist_ok=True)

# 참조 모델 리스트
reference_models = ["KCBERT", "KRFinBERT", "KBAlbert"]

# Ollama API를 사용하여 모델 이름 리스트 가져오기
models = ['llama3.1:8b', 'mistral:7b', 'llama3.2:3b', 'gemma2:27b']

def extract_sentiment(text):
    keywords = ["긍정", "부정", "중립"]
    text = str(text).lower()
    indices = {keyword: text.find(keyword) for keyword in keywords}
    earliest = min((index for index in indices.values() if index != -1), default=-1)
    for keyword, index in indices.items():
        if index == earliest:
            return keyword
    return "중립"

# 각 모델 및 참조 모델에 대해 sentiment 분석 수행
for model_name in models:
    sanitized_model_name = re.sub(r'[^a-zA-Z0-9_-]', '_', model_name)
    
    for ref_model in reference_models:
        # 참조 모델 데이터 파일 경로
        reference_file = os.path.join(data_folder, f'{ref_model}.csv')
        
        # 참조 모델 데이터 로드
        if os.path.exists(reference_file):
            ref_df = pd.read_csv(reference_file, encoding='utf-8-sig')
        else:
            ref_df = pd.DataFrame(columns=['Title', 'Predict'])  # 빈 데이터프레임 초기화

        # 모델별, 참조 모델별 CSV 파일 경로 설정
        output_csv = os.path.join(output_folder, f'{sanitized_model_name}_{ref_model}-BOOTICL.csv')
        
        # Load or create the DataFrame
        if os.path.exists(output_csv):
            news_data = pd.read_csv(output_csv, encoding='utf-8-sig')
        else:
            news_data = pd.read_csv(input_csv, encoding='utf-8-sig')
            news_data['Sentiment Analysis'] = None  # Initialize with None for new column

        # Loop through each article and ask for sentiment analysis only if 'Sentiment Analysis' is empty
        for index, row in news_data.iterrows():
            if pd.isna(row['Sentiment Analysis']):  # Check if the sentiment analysis is missing
                sentiments = []
                for _ in range(5):  # 5회 반복
                    ref_predict = None
                    ref_row = ref_df[ref_df['Title'] == row['Title']]
                    if not ref_row.empty:
                        ref_predict = ref_row.iloc[0]['Predict']
                    
                    # Define the system message to set the role
                    system_message = {
                        'role': 'system',
                        'content': "당신은 금융 뉴스 기사에서 감정을 평가하는 주식 분석가입니다."
                    }

                    # Define the question based on the article title and provide the response format
                    question = f"'{row['Title']}' 제목의 {row['Company']}에 대한 톤이 긍정적, 부정적 또는 중립적인지 결정하십시오.\n" \
                               f"참고로, {ref_model} 모델의 예측은 {ref_predict}입니다.\n" \
                               "다음 형식으로 응답해 주세요\n" \
                               "Sentiment: [긍정/부정/중립]"

                    # Send the question to the model
                    response = ollama.chat(model=model_name, messages=[
                        system_message,
                        {
                            'role': 'user',
                            'content': question,
                        },
                    ])

                    # Extract the response content
                    sentiment_response = response['message']['content'] if 'message' in response and 'content' in response['message'] else "중립"
                    sentiment = extract_sentiment(sentiment_response)
                    sentiments.append(sentiment)

                # Majority voting for sentiment
                sentiment_count = Counter(sentiments)
                majority_sentiment, count = sentiment_count.most_common(1)[0]
                final_sentiment = majority_sentiment if count >= 3 else "중립"

                # Update the sentiment analysis directly in the DataFrame
                news_data.at[index, 'Sentiment Analysis'] = final_sentiment

                # Save the DataFrame back to CSV after each response
                news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

        # Apply the sentiment extraction function to create the Predict column
        news_data['Predict'] = news_data['Sentiment Analysis']
        
        # Save the updated DataFrame with the Predict column back to the CSV file
        news_data.to_csv(output_csv, index=False, encoding='utf-8-sig')

print("모든 모델의 감정 분석이 완료되었습니다.")


메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KCBERT-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KRFinBERT-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.1:8b 모델의 감정 분석이 KBAlbert 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_1_8b_KBAlbert-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KCBERT-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KRFinBERT-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: mistral:7b 모델의 감정 분석이 KBAlbert 참조 기반으로 완료되었습니다. 파일은 'Sentiment/mistral_7b_KBAlbert-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.2:3b 모델의 감정 분석이 KCBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_2_3b_KCBERT-BOOTICL.csv'에 저장되었습니다.
메시지 전송 성공: llama3.2:3b 모델의 감정 분석이 KRFinBERT 참조 기반으로 완료되었습니다. 파일은 'Sentiment/llama3_2_3b_KRFinBERT-BOOTICL.csv'에 저장되었습니다.
